In [ ]:
# Step 1: Setup Environment
%env PYTHONPATH = # /env/python

!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
!conda update conda -y

import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

!conda create --name lsd-env python=3.7 -y

In [ ]:
# Step 2: Install dependencies/packages in the Conda environment and cloning lucid-sonic-dreams
%%shell
eval "$(conda shell.bash hook)"
conda activate lsd-env
pip install tensorflow==1.15
git clone https://github.com/nerdyrodent/lucid-sonic-dreams.git
cd lucid-sonic-dreams
python setup.py install
pip install tqdm
pip install scipy
pip install Pillow
pip install scikit-image
pip install librosa==0.8.0
pip install moviepy
pip install pygit2
pip install pandas
pip install gdown
pip install mega.py
pip install click
pip install psutil
pip uninstall torch torchvision -y
pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 opensimplex

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install Flask and pyngrok if not already installed
!pip install flask pyngrok --upgrade

# Authenticate ngrok with the correct authtoken

# Replace ngrok_auth_token with your actual authtoken from the ngrok dashboard

# Read the ngrok auth token from the txt file
with open('/content/drive/MyDrive/ngrok_token.txt', 'r') as file:
    ngrok_auth_token = file.read().strip()

# Authenticate ngrok with the token read from the file
!ngrok config add-authtoken {ngrok_auth_token}


import subprocess
from flask import Flask, request, jsonify, send_file
import threading
from pyngrok import ngrok
import os

app = Flask(__name__)

# Variable to store the path of the last uploaded audio file
last_uploaded_audio_file = None

@app.route('/run', methods=['POST'])
def run_lucidrun():
    global last_uploaded_audio_file

    # Check if the post request has the audio file
    if 'audio_file' in request.files:
        audio_file = request.files['audio_file']
        audio_file_path = f"/content/{audio_file.filename}"
        audio_file.save(audio_file_path)
        last_uploaded_audio_file = audio_file_path
    elif last_uploaded_audio_file:
        audio_file_path = last_uploaded_audio_file
    else:
        return jsonify({"status": "error", "message": "No audio file provided and no previously uploaded file found"}), 400
    data = request.json
    # Extract parameters from request form data
    style = data.get('style', '/content/drive/MyDrive/StyleGAN2/VisionaryArt.pkl')
    file_name = data.get('file_name', '/content/outputvideo.mp4')
    start = data.get('start', 0)
    duration = data.get('duration', 20)
    speed_fpm = data.get('speed_fpm', 20)
    pulse_percussive = data.get('pulse_percussive', True)
    pulse_harmonic = data.get('pulse_harmonic', False)
    pulse_react = data.get('pulse_react', 0.2)
    motion_harmonic = data.get('motion_harmonic', True)
    motion_percussive = data.get('motion_percussive', False)
    motion_react = data.get('motion_react', 0.2)
    fps = data.get('fps', 24)

    # Python code to be written to a new file
    python_code = f"""
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song='{audio_file_path}', style='{style}')

L.hallucinate(file_name='{file_name}',
              start={start},
              duration={duration},
              speed_fpm={speed_fpm},
              pulse_percussive={pulse_percussive},
              pulse_harmonic={pulse_harmonic},
              pulse_react={pulse_react},
              motion_harmonic={motion_harmonic},
              motion_percussive={motion_percussive},
              motion_react={motion_react},
              fps={fps})
"""

    # Write the Python code to a file
    script_path = '/content/lucid-sonic-dreams/run_lucid.py'
    with open(script_path, 'w') as file:
        file.write(python_code)

    # Construct command for running the script
    command = (
        'conda run -n lsd-env python /content/lucid-sonic-dreams/run_lucid.py'
    )

    try:
        # Run the command
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)

        # Check if the file was generated
        if os.path.exists(file_name):
            return send_file(file_name, as_attachment=True)
        else:
            return jsonify({"status": "error", "message": "File was not generated."}), 500

    except subprocess.CalledProcessError as e:
        return jsonify({"status": "error", "message": e.stderr}), 500


# Run the Flask App in a Thread
def run_flask():
    app.run(port=5002)  # Use a different port, 5002

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait a bit for the server to start
import time
time.sleep(5)

# Set up ngrok to expose the Flask server
public_url = ngrok.connect(5002)  # Match the port used in Flask
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5002\"")